In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
import csv
import matplotlib.pyplot as plt
import os
# API_KEY = **insert key here** 

stocks = ['LRCX', 'MDB', 'NOW', 'SNOW', 'CRWD', 'PANW', 'META', 'SPOT', 'ASML', 'NVDA', 'TSM', 'UBER', 'OTEX', 'ADBE', 'MSFT', 'ORCL', 'PLTR', 'SNPS', 'AAPL', 'GOOG', 'IBM', 'ANET', 'AVGO', 'CRM', 'KNSL']
print(len(stocks))

stock_names = ['Lam Research Corporation', 'MongoDB Inc.', 'ServiceNow Inc.', 'Snowflake Inc.', 'CrowdStrike Holdings Inc.', 'Palo Alto Networks Inc.', 'Meta Platforms Inc.', 'Spotify Technology S.A.', 'ASML Holding N.V.', 'NVIDIA Corporation', 'Taiwan Semiconductor Manufacturing Company Limited', 'Uber Technologies Inc.', 'Open Text Corporation', 'Adobe Inc.', 'Microsoft Corporation', 'Oracle Corporation', 'Palantir Technologies Inc.', 'Synopsys Inc.', 'Apple Inc.', 'Alphabet Inc.', 'International Business Machines Corporation', 'Arista Networks Inc.', 'Broadcom Inc.', 'Salesforce.com Inc.', 'Kinsale Capital Group Inc.']

25


In [5]:
def create_folders(parent_dir, folder_names):              # for creating directory structure
    for name in folder_names:
        # Construct the full path of the new folder
        folder_path = os.path.join(parent_dir, name)

        # Check if folder already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Folder '{name}' created at '{folder_path}'.")
        else:
            print(f"Folder '{name}' already exists at '{folder_path}'.")

#create_folders('./STOCKS_final/', stock_names)

Folder 'Lam Research Corporation' created at './STOCKS_final/Lam Research Corporation'.
Folder 'MongoDB Inc.' created at './STOCKS_final/MongoDB Inc.'.
Folder 'ServiceNow Inc.' created at './STOCKS_final/ServiceNow Inc.'.
Folder 'Snowflake Inc.' created at './STOCKS_final/Snowflake Inc.'.
Folder 'CrowdStrike Holdings Inc.' created at './STOCKS_final/CrowdStrike Holdings Inc.'.
Folder 'Palo Alto Networks Inc.' created at './STOCKS_final/Palo Alto Networks Inc.'.
Folder 'Meta Platforms Inc.' created at './STOCKS_final/Meta Platforms Inc.'.
Folder 'Spotify Technology S.A.' created at './STOCKS_final/Spotify Technology S.A.'.
Folder 'ASML Holding N.V.' created at './STOCKS_final/ASML Holding N.V.'.
Folder 'NVIDIA Corporation' created at './STOCKS_final/NVIDIA Corporation'.
Folder 'Taiwan Semiconductor Manufacturing Company Limited' created at './STOCKS_final/Taiwan Semiconductor Manufacturing Company Limited'.
Folder 'Uber Technologies Inc.' created at './STOCKS_final/Uber Technologies Inc

In [6]:
def create_csv_with_header(filename):           # for creating csv files
    # Define the header row
    header = [
        'price', 'price_change', 'price_change_percent', 'market_cap', 'PE', 'dividend_yield', 'eps',
        'week_high', 'week_low', 'beta', 'forward_pe', 'price_to_sales', 'price_to_book',
        'earnings_yield', 'latest_volume', 'average_volume', 'analyst_recommendation'
    ]

    # Write the header to the CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)

def main(folder, symbol):                  # Iterate over each stock symbol in a sector
    for file in folder:
        # Create a CSV file for the stock symbol
        filename = f"./STOCKS_final/{symbol}.csv"
        create_csv_with_header(filename)
        print(f"CSV file created for {symbol}")

#for i in range(len(stocks)):
    #main('./STOCKS_final', stocks[i])

CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for LRCX
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for MDB
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for NOW
CSV file created for SNOW
CSV file created for SNOW
CSV file created for SNOW
CSV file c

In [7]:
def save_data(symbol):
    ts = TimeSeries(key=API_KEY)
    fd = FundamentalData(key=API_KEY)

    # Get quote data
    data, meta_data = ts.get_quote_endpoint(symbol=symbol)

    if 'Error Message' in data:
        print("Error:", data['Error Message'])
        return None

    # Extracting data points
    price = float(data.get('05. price'))
    price_change = float(data.get('09. change'))
    price_change_percent = price_change / price * 100

    # Get company overview data
    data, meta_data = fd.get_company_overview(symbol=symbol)

    if 'Error Message' in data:
        print("Error:", data['Error Message'])
        return None

    market_cap = float(data.get('MarketCapitalization'))

    if data['PERatio'] == 'None': # Get the P/E ratio
        PE = 0
    else:                         # in case ratio not available
        PE = float(data['PERatio'])

    if data['DividendYield'] == 'None': # Get the P/E ratio
        dividend_yield = 0
    else:                         # in case ratio not available
        dividend_yield = float(data['DividendYield'])

    eps = float(data.get('EPS'))
    week_high = float(data.get('52WeekHigh'))
    week_low = float(data.get('52WeekLow', "N/A"))  # Provide a default value if key doesn't exist
    beta = float(data.get('Beta'))
    forward_pe = float(data.get('ForwardPE'))
    price_to_sales = float(data.get('PriceToSalesRatioTTM'))

    if data['PriceToBookRatio'] == 'None' or '-': # Get the P/E ratio
        price_to_book = 0
    else:                         # in case ratio not available
        price_to_book = float(data['PriceToBookRatio'])

    earnings_yield = eps / price * 100
    analyst_recommendation = data.get('AnalystRecommendation')

    # Retrieve volume and average volume data
    volume_data, _ = ts.get_daily(symbol=symbol)
    volumes = [int(volume_data[date]['5. volume']) for date in volume_data]
    average_volume = sum(volumes) / len(volumes)
    latest_volume = int(volume_data[max(volume_data.keys())]['5. volume'])

    # Saving data points into an array
    data_points = [
        price, price_change, price_change_percent, market_cap, PE, dividend_yield, eps,
        week_high, week_low, beta, forward_pe, price_to_sales, price_to_book,
        earnings_yield, latest_volume, average_volume, analyst_recommendation,
    ]

    print("Data points saved to array:", data_points)

    return data_points

def save_data_to_csv(symbol):
    filename = f"./STOCKS_final/{symbol}.csv"

    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(save_data(symbol))

    print(f"Data points saved to {filename}")

In [8]:
for i in range(len(stocks)):
    save_data_to_csv(stocks[i])
    print(i)

Data points saved to array: [870.25, -18.67, -2.1453605285837405, 114092384000.0, 33.57, 0.0092, 25.92, 1005.23, 488.2, 1.501, 26.25, 8.76, 0, 2.9784544671071536, 1354476, 1022390.22, None]
Data points saved to ./STOCKS_final/LRCX.csv
Data points saved to array: [327.47, -8.08, -2.4674015940391483, 23850002000.0, 0, 0, -2.48, 509.62, 212.52, 1.21, 142.86, 15.24, 0, -0.7573212813387485, 1353402, 1564457.12, None]
Data points saved to ./STOCKS_final/MDB.csv
Data points saved to array: [713.91, -17.45, -2.4442856942751887, 146624266000.0, 84.79, 0, 8.42, 815.32, 427.68, 0.957, 58.14, 17.6, 0, 1.1794203751173118, 1540609, 1156733.28, None]
Data points saved to ./STOCKS_final/NOW.csv
Data points saved to array: [145.45, -2.96, -2.035063595737367, 48609391000.0, 0, 0, -2.55, 237.72, 135.26, 0.907, 212.77, 18.88, 0, -1.7531797868683399, 4974560, 6470137.6, None]
Data points saved to ./STOCKS_final/SNOW.csv
Data points saved to array: [282.64, -11.46, -4.054627795075008, 68361855000.0, 763.89,

ValueError: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.